In [15]:
# assistant_protocol.py

from abc import ABC, abstractmethod
from typing import List

from openai.types.beta.threads.run import Run
from openai.types.beta.threads import ThreadMessage
from openai.types.beta.thread import Thread
from openai.pagination import SyncCursorPage


class AssistantBase(ABC):
    name: str
    description: str
    model: str
    instructions: str

    @abstractmethod
    def create_assistant(self, name: str, description: str, instructions: str) -> str:
        pass

    @abstractmethod
    def submit_message(self, assistant_id: str, thread: str, user_message: str) -> Run:
        pass

    @abstractmethod
    def submit_message(self, assistant_id: str, thread: str, user_message: str) -> Run:
        pass

    @abstractmethod
    def get_response(self, thread) -> SyncCursorPage[ThreadMessage]:
        pass

    @abstractmethod
    def create_thread_and_run(self, assistant_id: str, user_input: str):
        pass

    @abstractmethod
    def wait_on_run(self, run: Run, thread: Thread):
        pass

    @abstractmethod
    def display_message(self, messages: List[ThreadMessage]) -> None:
        pass

    async def new_thread_async(self) -> None:
        pass

    async def get_thread_async(self, thread_id: int) -> None:
        pass

In [6]:
# config_model.py

import yaml

class SKAssistantConfigModel:
    def __init__(self, name: str, description: str, instructions: str):
        self.name = name
        self.description = description
        self.instructions = instructions

    @classmethod
    def load_from_definition_string(cls, string: str) -> "SKAssistantConfigModel":
        yaml_data = yaml.safe_load(string)
        return cls(
            name=yaml_data['name'],
            description=yaml_data['description'],
            instructions=yaml_data['instructions']
        )

    @classmethod
    def load_from_definition_file(cls, path: str) -> "SKAssistantConfigModel":
        with open(path, 'r') as file:
            yaml_data = yaml.safe_load(file)
            return cls(
                name=yaml_data['name'],
                description=yaml_data['description'],
                instructions=yaml_data['instructions']
            )

In [17]:
# openai_helper.py

import openai
import time

from openai.types.beta.threads.run import Run
from openai.types.beta.threads import ThreadMessage
from openai.types.beta.thread import Thread
from openai.pagination import SyncCursorPage

class OpenAIAssistant(AssistantBase):
    def __init__(self, model):
        '''
        Initialize the OpenAI client based on the provided model.
        '''
        self.client = openai.OpenAI()
        self.model = model

    def create_assistant(self, name: str, description: str, instructions: str) -> str:
        '''
        Create an assistant with the provided name, description, and instructions.

        Args:
            name (str): The name of the assistant
            description (str): The description of the assistant
            instructions (str): The instructions of the assistant

        Returns:
            str: The ID of the created assistant
        '''
        assistant = self.client.beta.assistants.create(
            name=name,
            instructions=instructions,
            description=description,
            model=self.model,
        )
        return assistant.id
    

    def submit_message(self, assistant_id: str, thread: str, user_message: str) -> Run:
        '''
        Submit a message to the provided thread.

        Args:
            assistant_id (str): The ID of the assistant
            thread (str): The thread to submit the message to
            user_message (str): The message to submit

        Returns:
            Run: The run object
        '''
        self.client.beta.threads.messages.create(
            thread_id=thread.id, role="user", content=user_message
        )
        return self.client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant_id,
        )
    

    def get_response(self, thread) -> SyncCursorPage[ThreadMessage]:
        '''
        Get the response from the provided thread.

        Args:
            thread (str): The thread to get the response from

        Returns:
            SyncCursorPage[ThreadMessage]: The response from the thread
        '''
        return self.client.beta.threads.messages.list(thread_id=thread.id, order="asc")


    def create_thread_and_run(self, assistant_id: str, user_input: str):
        '''
        Create a thread and submit a message to it.

        Args:
            assistant_id (str): The ID of the assistant
            user_input (str): The message to submit

        Returns:
            Tuple[Thread, Run]: The thread and run objects
        '''
        thread = self.client.beta.threads.create()
        run = self.submit_message(assistant_id, thread, user_input)
        return thread, run


    def wait_on_run(self, run: Run, thread: Thread):
        '''
        Wait for the run to complete.

        Args:
            run (Run): The run object
            thread (Thread): The thread object

        Returns:
            Run: The updated run object
        '''
        while run.status == "queued" or run.status == "in_progress":
            run = self.client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id,
            )
            time.sleep(0.5)
        return run


    def display_message(self, messages: List[ThreadMessage]) -> None:
        '''
        Pretty print the provided messages.

        Args:
            messages (List[ThreadMessage]): The messages to pretty print
        '''
        print("# Messages")
        for m in messages:
            print(f"{m.role}: {m.content[0].text.value}")
        print() # separate it with a new line

In [18]:
# from config_model import SKAssistantConfigModel
# from assistant import Assistant
# from openai_integration import OpenAIHelper

# Load configuration
definition_str = """
name: Parrot
instructions: |
  Repeat the user message in the voice of a pirate and then end with a parrot sound.
description: A fun chat bot.
"""
assistant_config = SKAssistantConfigModel.load_from_definition_string(definition_str)
assert assistant_config, 'The assistant config cannot be empty'

# Initialize OpenAI Helper
oai_model = "gpt-3.5-turbo-1106"
oai_assistant = OpenAIAssistant(oai_model)
assistant_id = oai_assistant.create_assistant(
    name=assistant_config.name,
    instructions=assistant_config.instructions,
    description=assistant_config.description,
)

# Process messages
messages = [
    "Fortune favors the bold.",
    "I came, I saw, I conquered.",
    "Practice makes perfect."
]

thread_run_pairs = [oai_assistant.create_thread_and_run(assistant_id, message) for message in messages]

for thread, run in thread_run_pairs:
    run = oai_assistant.wait_on_run(run, thread)
    oai_assistant.display_message(oai_assistant.get_response(thread))

# Messages
user: Fortune favors the bold.
assistant: Fortune favors the bold, arrr! *parrot sound*

# Messages
user: I came, I saw, I conquered.
assistant: Arrr, I came, I saw, I conquered. *squawk*

# Messages
user: Practice makes perfect.
assistant: Arrr, practice makes perfect! *squawk*

